# Prerequisite

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_openai
!pip install langchainhub
!pip install langchain_community
!pip install datasets
!pip install transformers
!pip install langchain faiss-cpu
!pip install wikipedia
!pip install langchain-chroma
!pip install langchain_teddynote
!pip install pypdf
!pip install sentence_transformers
!pip install rank_bm25

In [ ]:
import pprint
import os
pp = pprint.PrettyPrinter(indent=4)

API_KEY="s"
LANGCHAIN_API_KEY=""

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

# Langchain basics

## Create a chat model

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-1106")

## Call the model using invoke

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

response = model.invoke([HumanMessage(content="Hi! I'm a student at Yonsei university")])
pp.pprint(response)

In [ ]:
print("Response")
pp.pprint(response.content)
print("="*10)
cost = response.usage_metadata['input_tokens'] * 1 + response.usage_metadata['output_tokens'] * 0.002
print(f"Cost: {cost}") # 요금계산

In [ ]:
## Multi-step interactions

messages = [
        SystemMessage(content="You are a helpful assistant. Answer all questions to the best of your ability."),
        HumanMessage(content="Hi! I'm a student at Yonsei university"),
        AIMessage(content="Nice to meet you! How can I assist you today?"),
        HumanMessage(content="Where am I studying now?"),
]

res = model.invoke(messages)
pp.pprint(res.content)

## PromptTemplate w/ chain

In [ ]:
from langchain_teddynote.messages import stream_response
from langchain.prompts import PromptTemplate


input_template = "What is the capital city of {country}?"

prompt = PromptTemplate(
    input_variables=["country"],
    template=input_template
)

## CHAIN ##
chain = prompt | model

In [ ]:
chain

In [ ]:
input = {"country": "Tai"}

res = chain.invoke(input)

pp.pprint(res.content)

In [ ]:
input = {"country": "Korea"}

res = chain.invoke(input)

pp.pprint(res.content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

template = """
You are an English teacher in your 10th year of teaching English. Please write an English conversation in [FORMAT] about the situation.

Situation:
{question}

FORMAT:
- English conversation:
- 한글 해석:
"""


prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [ ]:
chain1 = prompt | model
chain2 = prompt | model | output_parser

In [ ]:
input = {"question": "I want to go to a restaurant and order food"}
res1 = chain1.invoke(input)
res2 = chain2.invoke(input)

print(res1.content)
print("="*20)
print(res2)

In [ ]:
# For streaming
answer = chain1.stream(input)
stream_response(answer)

## Json parser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure.
class Conversation(BaseModel):
    english: str = Field(description="Full English conversation history")
    korean: str = Field(description="Full Korean conversation history")

## Define parser
json_parser = JsonOutputParser(pydantic_object=Conversation)



prompt = """
You are an English teacher in your 10th year of teaching English. Please write an English conversation in [FORMAT] about the situation.

Situation:
{question}

{format_instructions}
"""
prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt,
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)


## Define chain
chain = prompt | model | json_parser


In [ ]:
pp.pprint(json_parser.get_format_instructions())

In [ ]:
answer = chain.invoke(input)
print(f"type: {type(answer)}")
pp.pprint(answer)

## Create the ChatPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant. Answer all questions to the best of your ability."),
        ## List place holder ##
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model | StrOutputParser()

In [ ]:
messages = [HumanMessage(content="hi! I'm Taeyoon"), AIMessage(content="wassup"), HumanMessage(content="where is hospital?")]
response = chain.invoke({"messages": messages})

pp.pprint(response)

In [ ]:
# Other examples using placeholder
prompt = ChatPromptTemplate.from_messages(
    [
        ## System message with placeholder ##
        SystemMessagePromptTemplate.from_template("You are a helpful assistant. You should use {language} only."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm Taeyoon")], "language": "Korean"}
)

pp.pprint(response.content)

- Other examples

In [ ]:
system_template = "You are a helpful assistant that translate {input_language} to {output_language}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

input_dict = {
    "input_language":"English",
    "output_language":"Korean",
    "text":"I love I love this NLP course."
    }

print(chain.invoke(input_dict))

In [ ]:
#### My Example
#myprompt = HumanMessagePromptTemplate.from_template("promblem : {text}. you have to detail this problem and decsripte detail.")

myprompt = PromptTemplate(input_variables=["text"], template="promblem : {text}. you have to detail this problem and decsripte detail.")

myai = model | output_parser

print(myai.invoke([SystemMessage(content="You are a helpful assistant that translate enlish to korea"), HumanMessage(content="promblem : 3+2*(4+7)=?. you have to detail this problem and decsripte detail.")]))
#print(myai.invoke({"text" : "3+2*(4+7)=?"}))

In [ ]:
#### My Example
#myprompt = PromptTemplate(input_variables=["text"], template="promblem : {text}. you have to detail this problem and decsripte detail.")
myprompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are a korean assistant. only answer to use korean"),
        HumanMessagePromptTemplate.from_template("promblem : {text}. you have to detail this problem and decsripte detail.")
    ]
)

myai = myprompt | model | output_parser

#"text"="3+2*(4+7)=?"
#messages = [SystemMessage(content="You are a helpful assistant that translate enlish to korea"), HumanMessage(content="3+2*(4+7)=?")]
print(myai.invoke({"text":"3+2*(4+7)"}))
#print(myai.invoke({"text" : "3+2*(4+7)=?"}))

# Retrieval Details

In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

## Save articles from wikipedia

In [ ]:
import wikipedia

# Fetch the NLP article
article_name = "Chelsea Football Club"
article = wikipedia.page(article_name)
article_save_name = "chelsea"

pp.pprint(article.content)

# Save the content to a file
with open(f"{article_save_name}.txt", "w", encoding="utf-8") as f:
    f.write(article.content)

## Quick practice

In [ ]:
## Document loader
loader = TextLoader("chelsea.txt")
documents = loader.load()

## Text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Embedding
embeddings = OpenAIEmbeddings(api_key=API_KEY)

## vectortstore
vectorstore = FAISS.from_documents(texts, embeddings)

## retriever
retriever = vectorstore.as_retriever()

In [ ]:
## retrieve
query = "When did chelsea won the first champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("!@!@!"*20)

## TF-IDF & BM-25

In [ ]:
## Document loader
loader = TextLoader("chelsea.txt")
documents = loader.load()

## Text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator='\n')
texts = text_splitter.split_documents(documents)

In [ ]:
from langchain_community.retrievers import TFIDFRetriever, BM25Retriever
## TF-IDF & BM25
retriever1 = TFIDFRetriever.from_documents(texts)
retriever2 = BM25Retriever.from_documents(texts)

In [ ]:
## retrieve
query = "When did chelsea won the first champions league title?"
results = retriever1.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

In [ ]:
## retrieve
query = "When did chelsea won the first champions league title?"
results = retriever2.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

## Details; ablation study & practices

1. Document loader
2. Embedding models
3. Vector stores
4. Retrieval



### Document loader

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Ch11-Reasoning.pdf")
pages = loader.load_and_split()

## Embedding
embeddings = OpenAIEmbeddings(api_key=API_KEY)

## vectortstore
vectorstore = FAISS.from_documents(pages, embeddings)

## retriever
retriever = vectorstore.as_retriever()

In [ ]:
pages[1]

In [ ]:
## retrieve
query = "What is commonsense reasoning?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

### Embedding models

#### in-depth

In [ ]:
embeddings_model = OpenAIEmbeddings()

documents = [
    "Hi there!",
    "Oh, hello!",
    "What's your name?",
    "My friends call me Connor",
    "Hello World!"
]
embeddings = embeddings_model.embed_documents(documents)
len(embeddings), len(embeddings[0])

In [ ]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
len(embedded_query)

In [ ]:
import numpy as np


# Convert embeddings to numpy arrays
document_embeddings = np.array(embeddings)
query_embedding = np.array(embedded_query)

# Compute cosine similarities using NumPy broadcasting
# Normalize the document embeddings
norm_document_embeddings = np.linalg.norm(document_embeddings, axis=1)
normalized_document_embeddings = document_embeddings / norm_document_embeddings[:, np.newaxis]

# Normalize the query embedding
norm_query_embedding = np.linalg.norm(query_embedding)
normalized_query_embedding = query_embedding / norm_query_embedding


# Compute the dot product between the query embedding and each document embedding
similarities = np.dot(normalized_document_embeddings, normalized_query_embedding)

# Print the length of the query embedding and the similarity scores
print("Length of the query embedding:", len(embedded_query))
print("Cosine similarities:", similarities)

# Optionally, display the document with the highest similarity
most_similar_index = np.argmax(similarities)
print("Most similar document:", documents[most_similar_index])

#### Open-source embedding models

In [ ]:
## Document loader
loader = TextLoader("chelsea.txt")
documents = loader.load()

## Text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Embedding

model_name1 = "facebook/contriever"
model_name2 = "all-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(model_name=model_name1)
## vectortstore
vectorstore = FAISS.from_documents(texts, embeddings)

## retriever
retriever = vectorstore.as_retriever()


In [ ]:
## retrieve
query = "When did chelsea won the first champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

### Vector stores

In [ ]:
from langchain_chroma import Chroma

## Document loader
loader = TextLoader("chelsea.txt")
documents = loader.load()

## Text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Embedding
embeddings = OpenAIEmbeddings()

## vectortstore
# vectorstore = FAISS.from_documents(texts, embeddings)
vectorstore = Chroma.from_documents(texts, embeddings)

## retriever
retriever = vectorstore.as_retriever()

In [ ]:
## retrieve
query = "When did chelsea won the first champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

### Retrieval

#### algorithms

In [ ]:
### Settings

## Document loader
loader = TextLoader("chelsea.txt")
documents = loader.load()

## Text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Embedding
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

## vectortstore
vectorstore = FAISS.from_documents(texts, embeddings)

In [ ]:
### Basic
## retriever
retriever = vectorstore.as_retriever()

## retrieve
query = "When did chelsea won the champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

In [ ]:
### Searchtype: MMR (Maximum marginal relevance retrieval)
## retriever
retriever = vectorstore.as_retriever(search_type="mmr")

## retrieve
query = "When did chelsea won the champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

In [ ]:
### Score_threshold
## retriever
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold": 0.5})

## retrieve
query = "When did chelsea won the champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

In [ ]:
### Top k
## retriever
retriever = vectorstore.as_retriever(search_kwargs={"top_k": 3})

## retrieve
query = "When did chelsea won the champions league title?"
results = retriever.invoke(query)

for doc in results:
    pp.pprint(doc.page_content)
    print("-"*20)

#### MultiQueryRetriever

In [ ]:
## Document loader
loader = TextLoader("chelsea.txt")
documents = loader.load()

## Text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Embedding
embeddings = OpenAIEmbeddings(api_key=API_KEY)

## vectortstore
vectorstore = Chroma.from_documents(texts, embeddings)

## retriever
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "When did chelsea won the champions league title?"

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0.7)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever, llm=llm
)

In [ ]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
unique_docs = retriever_from_llm.invoke(question)
print(len(unique_docs))

In [ ]:
for doc in unique_docs:
    pp.pprint(doc.page_content)
    print("-"*20)

# RAG

## Basic RAG application

In [ ]:
import getpass
from langchain_openai import ChatOpenAI
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

### Retriever settings

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.0)

In [ ]:
# Load the PDF file.
file_path = (
    "./Ch11-Reasoning.pdf"
)
loader = PyPDFLoader(file_path)
PDF = loader.load_and_split()

In [ ]:
print(len(PDF))

In [ ]:
pp.pprint(PDF[2])

In [ ]:
# Indexing: Split the File
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(PDF)

# Indexing
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# Retrieve the pages in PDF
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("What is commonsense reasoning?")

len(retrieved_docs)

### generation setting

In [ ]:
# Generate answer
from langchain import PromptTemplate

prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [ ]:
# Defining the chain by LCEL Runnable protocol
def format_docs(PDF):
    return "\n\n".join(doc.page_content for doc in PDF)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
query = "What is commonsense reasoning?"
res = rag_chain.invoke(query)
pp.pprint(res)

### Practice

In [ ]:
class RAG:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.0)
        self.embedding = OpenAIEmbeddings()

    def get_pdf(self, file_path):
        loader = PyPDFLoader(file_path)
        PDF = loader.load_and_split()
        # Indexing: Split the File
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )
        all_splits = text_splitter.split_documents(PDF)

        # Indexing
        vectorstore = FAISS.from_documents(documents=all_splits, embedding=self.embedding)
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        return PDF

    def get_txt(self, file_path):
        loader = TextLoader(file_path)
        txt = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )
        all_splits = text_splitter.split_documents(txt)

        vectorstore = FAISS.from_documents(documents=all_splits, embedding=self.embedding)
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        return txt

    def set_prompt(self, prompt_template, input_variables):
        self.prompt = PromptTemplate(
            input_variables=input_variables,
            template=prompt_template,
        )
        return True

    def format_docs(self, doc):
        return "\n\n".join(doc.page_content for doc in doc)

    def generate_answer(self, question):
        rag_chain = (
            {"context": self.retriever | self.format_docs, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
        res = rag_chain.invoke(query)
        return res

In [ ]:
rag = RAG()

## get pdf
PDF = rag.get_pdf("./Ch11-Reasoning.pdf")

In [ ]:
## set prompt
prompt = """
당신은 질문 답변 작업을 돕는 비서입니다.
다음의 제공된 정보를 사용하여 질문에 답하십시오.
답을 모르면 모른다고 말하세요.
세 문장 이내로 답변을 간결하게 유지하십시오.
답변은 한국어로 작성해줘.

질문: {question}
정보: {context}
답변:"""
rag.set_prompt(prompt, ["context", "question"])

In [ ]:
## generate answer
query = "commonsense reasoning 이란 뭐야?"
res = rag.generate_answer(query)
pp.pprint(res)

## Conversational RAG application

In [ ]:
input = __builtins__.input

In [ ]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
PDF[0]

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.7)

# Load the PDF file.
file_path = (
    "./Ch11-Reasoning.pdf"
)
loader = PyPDFLoader(file_path)
PDF = loader.load_and_split()

# Indexing
vectorstore = FAISS.from_documents(documents=PDF, embedding=OpenAIEmbeddings())

# Retrieve the pages in PDF
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
system_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, say that you don't know.
Use three sentences maximum and keep the answer concise.

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is commonsense reasoning?"})
print(response.keys())

In [ ]:
pp.pprint(response["context"])

In [ ]:
pp.pprint(response["answer"])

### Contextualizing the questions

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = """
Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history.
Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
## Create a RAG chain with chat history

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
## Test
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is commonsense reasoning?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

print(ai_msg_1.keys())
print(len(ai_msg_1["context"]))
pp.pprint(ai_msg_1["answer"])

In [ ]:
pp.pprint(ai_msg_1["chat_history"])

In [ ]:
for doc in ai_msg_1["context"]:
    pp.pprint(doc.page_content)
    print("-"*20)

In [ ]:
second_question = "Tell me some benchmarks of it."
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

pp.pprint(ai_msg_2["answer"])

In [ ]:
docs = ai_msg_2["context"]

for d in docs:
    print(d.page_content)
    print("-"*20)

### Practice

In [ ]:
class RAGChatbot:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.0)
        self.embedding = OpenAIEmbeddings()
        self.context_history = []
        self.chat_history = []

    def get_pdf(self, file_path):
        loader = PyPDFLoader(file_path)
        PDF = loader.load_and_split()
        # Indexing: Split the File
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )
        all_splits = text_splitter.split_documents(PDF)

        # Indexing
        vectorstore = FAISS.from_documents(documents=all_splits, embedding=self.embedding)
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        return PDF

    def get_txt(self, file_path):
        loader = TextLoader(file_path)
        txt = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, add_start_index=True
        )
        all_splits = text_splitter.split_documents(txt)

        vectorstore = FAISS.from_documents(documents=all_splits, embedding=self.embedding)
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
        return txt

    def setup(self):
        contextualize_q_system_prompt = """
        Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history.
        Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
        """

        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        history_aware_retriever = create_history_aware_retriever(
            self.llm, self.retriever, contextualize_q_prompt
        )
        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )


        question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

        self.rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
        return True

    def chat(self):
        self.chat_history = []
        self.context_history = []
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit", "bye"]:
                print("Chatbot: Goodbye!")
                break
            try:
                ai_msg = self.rag_chain.invoke({"input": user_input, "chat_history": chat_history})
                self.chat_history = ai_msg["chat_history"]
                self.context_history.append(ai_msg["context"])
                pp.pprint(f"Chatbot: {ai_msg['answer']}")
            except Exception as e:
                print(f"Error: {str(e)}")

    def show_context(self):
        for idx, context in enumerate(self.context_history):
            print(f"============ Context for {idx} query ============")
            for doc in context:
                print(doc.page_content)
                print("-"*20)

In [ ]:
rag = RAGChatbot()

pdf = rag.get_pdf("./Ch11-Reasoning.pdf")

rag.setup()

In [ ]:
rag.chat()

In [ ]:
rag.show_context()